In [ ]:
from keras.models import Model
from keras.layers import Input, Dense, Conv2D, MaxPooling2D, UpSampling2D

import numpy as np
from matplotlib import pyplot as plt

import os
from PIL import Image

# The image is actually 240 x 416, 
#     so we will have 16 x 16 patches with frame shape of 15 x 26 instead of 26 x 15
data1_path = 'RaceHorses_416x240_30' 

In [ ]:
df =[]
n_images = len(os.listdir(data1_path))
for im_name in os.listdir(data1_path):
    input_im =  np.array(Image.open(data1_path + '/' + im_name))[...,:3]
    df.append(input_im)
df = np.array(df)
train_num = df.shape[0] * 3//4
np.random.shuffle(df)
train_data = df[:train_num]
test_data = df[train_num:]

In [ ]:
x=df.shape[1]
y=df.shape[2]
z=df.shape[3]

input_img = Input(shape=(x,y,z))

e = Conv2D(64, (7, 7), activation='relu', padding='same')(input_img)
e = MaxPooling2D((2, 2), padding='same')(e)
e = Conv2D(32, (5, 5), activation='relu', padding='same')(e)
e = MaxPooling2D((2, 2), padding='same')(e)
e = Conv2D(16, (1, 1), activation='relu', padding='same')(e)
e = Conv2D(8, (3, 3), activation='relu', padding='same')(e)
encoded = Conv2D(4, (3, 3), activation='relu', padding='same')(e)

d = Conv2D(4, (3, 3), activation='relu', padding='same')(encoded)
d = Conv2D(8, (3, 3), activation='relu', padding='same')(d)
d = Conv2D(16, (1, 1), activation='relu', padding='same')(d)
d = UpSampling2D((2, 2))(d)
d = Conv2D(32, (5, 5), activation='relu', padding='same')(d)
d = UpSampling2D((2, 2))(d)
d = Conv2D(64, (7, 7), activation='relu', padding='same')(d)
decoded = Conv2D(3, (1, 1), activation='linear')(d)

autoencoder = Model(input_img, decoded)
autoencoder.summary()
autoencoder.compile(optimizer='adam', loss='mse')

autoencoder.fit(train_data,train_data, epochs=10, batch_size=25)